# 12-Class SSVEP EEG Dataset - Classification Using Convolutional Neural Network
User-Dependent Training using Magnitude Spectrum Features and Complex Spectrum Features
(10-Fold Cross-validation)

Following implementation is an asynchronous SSVEP BCI using Convolutional Neural Network classification for 1 second data length.

Reference Paper: [Comparing user-dependent and user-independent training of CNN for SSVEP BCI](https://iopscience.iop.org/article/10.1088/1741-2552/ab6a67)


In [88]:
import sys
import os

sys.path.insert(0, os.path.abspath('..'))

%config Completer.use_jedi = False

In [89]:
%%capture
%load_ext autoreload
%autoreload 2
import warnings
import numpy as np
import numpy.matlib as npm
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sio
import pickle

import matplotlib.pyplot as plt

from keras.utils.np_utils import to_categorical
from keras import optimizers
from keras.losses import categorical_crossentropy

from bcilib.ssvep_utils_pytorch import CNN, RasterizeSlice, CustomTensorDataset
from bcilib import ssvep_utils as su 
from torch.utils.data import TensorDataset, DataLoader, random_split

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
import torch
import tqdm

warnings.filterwarnings('ignore')

In [90]:
example_model = CNN()
from torchsummary import summary
input_shape = (8, 220, 1)
summary(example_model.network.cuda(), input_shape)

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 16, 213, 1]          1,024
├─BatchNorm2d: 1-2                       [-1, 16, 213, 1]          32
├─ReLU: 1-3                              [-1, 16, 213, 1]          --
├─Dropout: 1-4                           [-1, 16, 213, 1]          --
├─Conv2d: 1-5                            [-1, 16, 204, 1]          2,560
├─BatchNorm2d: 1-6                       [-1, 16, 204, 1]          32
├─ReLU: 1-7                              [-1, 16, 204, 1]          --
├─Dropout: 1-8                           [-1, 16, 204, 1]          --
├─Flatten: 1-9                           [-1, 3264]                --
├─Linear: 1-10                           [-1, 12]                  39,168
Total params: 42,816
Trainable params: 42,816
Non-trainable params: 0
Total mult-adds (M): 0.78
Input size (MB): 0.01
Forward/backward pass size (MB): 0.10
Params size (MB): 0.16
Estimated Total Size (MB): 0.27


Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 16, 213, 1]          1,024
├─BatchNorm2d: 1-2                       [-1, 16, 213, 1]          32
├─ReLU: 1-3                              [-1, 16, 213, 1]          --
├─Dropout: 1-4                           [-1, 16, 213, 1]          --
├─Conv2d: 1-5                            [-1, 16, 204, 1]          2,560
├─BatchNorm2d: 1-6                       [-1, 16, 204, 1]          32
├─ReLU: 1-7                              [-1, 16, 204, 1]          --
├─Dropout: 1-8                           [-1, 16, 204, 1]          --
├─Flatten: 1-9                           [-1, 3264]                --
├─Linear: 1-10                           [-1, 12]                  39,168
Total params: 42,816
Trainable params: 42,816
Non-trainable params: 0
Total mult-adds (M): 0.78
Input size (MB): 0.01
Forward/backward pass size (MB): 0.10
Params size (MB): 0.16
Estimated Total Size (MB): 0.27

In [91]:
def get_training_data(features_data):
    features_data = np.reshape(features_data, (features_data.shape[0], features_data.shape[1], 
                                               features_data.shape[2], 
                                               features_data.shape[3]*features_data.shape[4]))
    train_data = features_data[:, :, 0, :].T
    for target in range(1, features_data.shape[2]):
        train_data = np.vstack([train_data, np.squeeze(features_data[:, :, target, :]).T])

    train_data = np.reshape(train_data, (train_data.shape[0], train_data.shape[1], 
                                         train_data.shape[2], 1))
    total_epochs_per_class = features_data.shape[3]
    features_data = []
    class_labels = np.arange(CNN_PARAMS['num_classes'])
    labels = (npm.repmat(class_labels, total_epochs_per_class, 1).T).ravel()
    labels = to_categorical(labels)
    
    return train_data, labels

In [92]:
def train_CNN_cross_val_predict(model, train_data, labels, num_folds=10):
    print("INPUT SHAPE: ", train_data.shape)
    
    labels = np.argmax(labels, axis=1)
    print("LABEL SHAPE: ", labels.shape)
    
    # frame_dataset = ShapesNpzDataset("/home/nogay/Desktop/frame_dataset/dataset.npz", target_transform=int)
    train_size = int(0.6 * len(train_data))
    val_size = int(0.2 * len(train_data))
    test_size = len(train_data) - train_size - val_size
    print(train_size, val_size, test_size)
    
    
    tensor_x = torch.from_numpy(train_data) # transform to torch tensor
    tensor_y = torch.from_numpy(labels.astype(int))
    
    print(tensor_x.dtype)
    print(tensor_y.dtype)

    dataset = TensorDataset(tensor_x, tensor_y) # create your datset
    
    train_dataset, val_dataset, test_dataset = random_split(
        dataset,
        [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )
    
    dataloader_train = DataLoader(train_dataset, shuffle=True, num_workers=4, batch_size=16)
    dataloader_val = DataLoader(val_dataset, shuffle=False, num_workers=4, batch_size=16)
    dataloader_test = DataLoader(test_dataset, shuffle=False, num_workers=4, batch_size=16)
    
    early_stopping = EarlyStopping('train_loss', patience=10, mode='min')
    trainer = pl.Trainer(gpus=1, 
                         max_epochs=200, 
                         enable_pl_optimizer=True,
                         callbacks=[early_stopping])
    
    trainer.fit(model, dataloader_train, val_dataloaders=dataloader_val)
    return trainer.test(model, test_dataloaders=dataloader_test, ckpt_path=None)
    #return cv_acc

In [93]:
data_path = os.path.abspath('data/original')

CNN_PARAMS = {
    'batch_size': 64,
    'epochs': 50,
    'droprate': 0.25,
    'learning_rate': 0.001,
    'lr_decay': 0.0,
    'l2_lambda': 0.0001,
    'momentum': 0.9,
    'kernel_f': 10,
    'n_ch': 8,
    'num_classes': 12}

FFT_PARAMS = {
    'resolution': 0.2930,
    'start_frequency': 3.0,
    'end_frequency': 35.0,
    'sampling_rate': 256
}

window_len = 1
shift_len = 1
    
all_acc = np.zeros((10, 1))

magnitude_spectrum_features = dict()
complex_spectrum_features = dict()

In [94]:
mcnn_training_data = dict()
ccnn_training_data = dict()

In [95]:
mcnn_results = dict()
ccnn_results = dict()

# Load Dataset and Segment

In [96]:
all_segmented_data = dict()
for subject in range(0, 10):
    dataset = sio.loadmat(f'{data_path}/s{subject+1}.mat')
    eeg = np.array(dataset['eeg'], dtype='float32')
    
    CNN_PARAMS['num_classes'] = eeg.shape[0]
    CNN_PARAMS['n_ch'] = eeg.shape[1]
    total_trial_len = eeg.shape[2]
    num_trials = eeg.shape[3]
    sample_rate = 256

    filtered_data = su.get_filtered_eeg(eeg, 6, 80, 4, sample_rate)
    all_segmented_data[f's{subject+1}'] = su.get_segmented_epochs(filtered_data, window_len, 
                                                                  shift_len, sample_rate)

# Feature Extraction

In [97]:
for subject in all_segmented_data.keys():
#     magnitude_spectrum_features[subject] = su.magnitude_spectrum_features(all_segmented_data[subject], 
#                                                                           FFT_PARAMS)
    complex_spectrum_features[subject] = su.complex_spectrum_features(all_segmented_data[subject], 
                                                                      FFT_PARAMS)

In [98]:
for subject in all_segmented_data.keys():
#    mcnn_training_data[subject] = dict()
    ccnn_training_data[subject] = dict()
#     train_data, labels = get_training_data(magnitude_spectrum_features[subject])
#     mcnn_training_data[subject]['train_data'] = train_data
#     mcnn_training_data[subject]['label'] = labels
    
    train_data, labels = get_training_data(complex_spectrum_features[subject])
    
    print(train_data.max())
    print(train_data.min())
    
    train_data = np.power(((train_data - train_data.min() + 1) / (train_data.max() - train_data.min())) * 100, 3)
    
    print(train_data.max())
    print(train_data.min())
    print()
    
    # print(train_data)
    
    ccnn_training_data[subject]['train_data'] = train_data
    ccnn_training_data[subject]['label'] = labels

4.453436744813788
-4.542808724004553
1371913.8662493997
1373.4602953441113

4.697890353148152
-5.587120200892877
1320966.1341687553
919.1486950344613

4.24294748232967
-3.9931721341508273
1410264.9429553908
1789.9135828709188

6.390093494479364
-7.7297572992609656
1227869.4496460753
355.23003409219797

21.082749462733876
-18.468665927927102
1077784.5713728983
16.162699947894495

6.618154987289777
-5.988042252342181
1257355.231914684
499.1691147218926

4.193549156789444
-4.533442971943087
1384656.1550238773
1504.5486911991609

4.8637124388793245
-5.111515517334839
1331901.6618140617
1007.468584966449

4.7738970955208275
-4.780804938509801
1347989.4343764812
1146.4326759918458

6.531397488715976
-6.482075939864547
1248698.7994694596
453.75383755283235



# M-CNN Training and Results

In [99]:
# acc_list = []

# for subject in mcnn_training_data.keys():
#     print(f'\nMCNN - Subject: {subject}')
#     train_data = mcnn_training_data[subject]['train_data']
#     labels = mcnn_training_data[subject]['label']
    
#     acc_list.append(train_CNN_cross_val_predict(train_data, labels, 10))

In [100]:
# l = 0
# for t in acc_list:
#     l += t[0]["test_acc"]
# print(l/len(acc_list))

# C-CNN Training and Results

In [101]:
c_acc_list = []
model_list = []

for subject, i in zip(ccnn_training_data.keys(), range(len(ccnn_training_data))):
    print(f'\nCCNN - Subject: {subject}')
    train_data = ccnn_training_data[subject]['train_data']
    labels = ccnn_training_data[subject]['label']
    m = CNN()
    model_list.append(m)
    c_acc_list.append(train_CNN_cross_val_predict(m, train_data, labels, 10))
    torch.save(m, "models/cnn-s{}.h5".format(i))



CCNN - Subject: s1
INPUT SHAPE:  (720, 8, 220, 1)
LABEL SHAPE:  (720,)
432 144 144
torch.float64
torch.int64



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.5833333333333334, 'test_loss': 1.464097499847412}
--------------------------------------------------------------------------------

CCNN - Subject: s2
INPUT SHAPE:  (720, 8, 220, 1)
LABEL SHAPE:  (720,)
432 144 144
torch.float64
torch.int64



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.3194444444444444, 'test_loss': 2.8520171642303467}
--------------------------------------------------------------------------------

CCNN - Subject: s3
INPUT SHAPE:  (720, 8, 220, 1)
LABEL SHAPE:  (720,)
432 144 144
torch.float64
torch.int64



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.8680555555555556, 'test_loss': 0.43123769760131836}
--------------------------------------------------------------------------------

CCNN - Subject: s4
INPUT SHAPE:  (720, 8, 220, 1)
LABEL SHAPE:  (720,)
432 144 144
torch.float64
torch.int64



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.9791666666666666, 'test_loss': 0.09876769036054611}
--------------------------------------------------------------------------------

CCNN - Subject: s5
INPUT SHAPE:  (720, 8, 220, 1)
LABEL SHAPE:  (720,)
432 144 144
torch.float64
torch.int64


...


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.9791666666666666, 'test_loss': 0.0867108553647995}
--------------------------------------------------------------------------------

CCNN - Subject: s7
INPUT SHAPE:  (720, 8, 220, 1)
LABEL SHAPE:  (720,)
432 144 144
torch.float64
torch.int64



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.8055555555555556, 'test_loss': 0.852876603603363}
--------------------------------------------------------------------------------

CCNN - Subject: s8
INPUT SHAPE:  (720, 8, 220, 1)
LABEL SHAPE:  (720,)
432 144 144
torch.float64
torch.int64



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.9791666666666666, 'test_loss': 0.09078437834978104}
--------------------------------------------------------------------------------

CCNN - Subject: s9
INPUT SHAPE:  (720, 8, 220, 1)
LABEL SHAPE:  (720,)
432 144 144
torch.float64
torch.int64



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.9305555555555556, 'test_loss': 0.2712527811527252}
--------------------------------------------------------------------------------

CCNN - Subject: s10
INPUT SHAPE:  (720, 8, 220, 1)
LABEL SHAPE:  (720,)
432 144 144
torch.float64
torch.int64



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.8194444444444444, 'test_loss': 0.6055747270584106}
--------------------------------------------------------------------------------


In [102]:
# # Start tensorboard.
# %load_ext tensorboard
# %tensorboard --logdir lightning_logs/

In [103]:
l = 0
for t in c_acc_list:
    print(t[0]["test_acc"])
    l += t[0]["test_acc"]
print("Overall ACC: ", l/len(c_acc_list))

0.5833333333333334
0.3194444444444444
0.8680555555555556
0.9791666666666666
0.9861111111111112
0.9791666666666666
0.8055555555555556
0.9791666666666666
0.9305555555555556
0.8194444444444444
Overall ACC:  0.825
